In [1]:
#Installing Kaggle
!pip install kaggle

In [2]:
#Configuring the kaggle.json file

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [3]:
#Fetching the data
!kaggle datasets download -d ekrembayar/fifa-21-complete-player-dataset

Dataset URL: https://www.kaggle.com/datasets/ekrembayar/fifa-21-complete-player-dataset
License(s): other
fifa-21-complete-player-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
#Extracting the compressed dataset
from zipfile import ZipFile
data = '/content/fifa-21-complete-player-dataset.zip'

with ZipFile(data, 'r') as zip:
  zip.extractall()
  print('Extraction Successful')

Extraction Successful


In [5]:
#Import the dependencies

import pandas as pd

In [6]:
df = pd.read_csv('/content/fifa21_male2.csv')
df.head()

<ipython-input-6-754ff7f4dff7>:1: DtypeWarning: Columns (78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/fifa21_male2.csv')


,ID,Name,Age,OVA,Nationality,Club,BOV,BP,Position,Player Photo,...,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,Gender
0,2,G. Pasquale,33,69,Italy,Udinese,71,LWB,LM,https://cdn.sofifa.com/players/000/002/16_120.png,...,70+-1,70+-1,71+-2,70+-1,69+0,69+0,69+0,70+-1,17+0,Male
1,16,Luis García,37,71,Spain,KAS Eupen,70,CM,CM CAM CDM,https://cdn.sofifa.com/players/000/016/19_120.png,...,66+1,66+1,62+1,60+1,60+1,60+1,60+1,60+1,17+1,Male
2,27,J. Cole,33,71,England,Coventry City,71,CAM,CAM RM RW LM,https://cdn.sofifa.com/players/000/027/16_120.png,...,54+0,54+0,52+0,47+0,46+0,46+0,46+0,47+0,15+0,Male
3,36,D. Yorke,36,68,Trinidad &amp; Tobago,Sunderland,70,ST,NaN,https://cdn.sofifa.com/players/000/036/09_120.png,...,65+0,65+0,56+0,57+0,51+0,51+0,51+0,57+0,22+0,Male
4,41,Iniesta,36,81,Spain,Vissel Kobe,82,CAM,CM CAM,https://cdn.sofifa.com/players/000/041/20_120.png,...,73+3,73+3,70+3,67+3,64+3,64+3,64+3,67+3,17+3,Male


In [7]:
#Removing truncation from the rows and columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [8]:
#Removing the lbs from the weight
df.Weight = df['Weight'].str.replace('lbs', '').astype(float)

#Converting the weight from lbs to kg
conversion = 0.453592

df['Weight (Kg)'] = df['Weight'] * conversion
df['Weight (Kg)'] = df['Weight (Kg)'].astype(int)
df.drop(columns=['Weight'], inplace=True)


In [9]:
#Cleaning the Height Column

#Extract the feet and inches
df['feet'] = df['Height'].str.extract(r"(\d+)'")[0].astype(float)
df['inches'] = df['Height'].str.extract(r"(\d+)\"")[0].astype(float)

#Conversion

foot_to_cm = 30.48
inch_to_cm = 2.54

#Convert the height to cm
df['Height (cm)'] = df['feet'] * foot_to_cm + df['inches'] * inch_to_cm
df['Height (cm)'] = df['Height (cm)'].astype(int)
df.drop(columns=['Height'], inplace=True)

In [10]:
#Splitting the Date

#Set the date column to the datetime format
df['Joined'] = pd.to_datetime(df['Joined'])

#Extract the year, month and day
df['Year'] = df['Joined'].dt.year
df['Month'] = df['Joined'].dt.month
df['Day'] = df['Joined'].dt.day

#Filling the null columns with placeholder values

df['Year'] = df['Year'].fillna(0).astype(int)
df['Month'] = df['Month'].fillna(0).astype(int)
df['Day'] = df['Day'].fillna(0).astype(int)

In [12]:
#Cleaning the Value, Wage and Release Clause Columns

def convert_amount(value):
  if 'K' in value:
    return float(value.replace('€', '').replace('K','')) * 1e3
  elif 'M' in value:
    return float(value.replace('€', '').replace('M', '')) * 1e6
  else:
    return float(value.replace('€', ''))

#Applying the conversion to the relevant columns

df['Value'] = df['Value'].apply(convert_amount)
df['Wage'] = df['Wage'].apply(convert_amount)
df['Release Clause'] = df['Release Clause'].apply(convert_amount)

In [14]:
# Split the 'Team & Contract' column into 'Team' and 'Contract Period' columns
df[['Team', 'Contract Period']] = df['Team & Contract'].str.extract(r'(.+?)\s+(\d{4}\s*~\s*\d{4})')

# Split the 'Contract Period' column into 'Date Started' and 'Date Ended'
df[['Date Started', 'Date Ended']] = df['Contract Period'].str.split('~', expand=True)

# Trim any extra whitespace from the start and end dates
df['Date Started'] = df['Date Started'].str.strip()
df['Date Ended'] = df['Date Ended'].str.strip()

In [16]:
df=df.drop(['Contract Period', 'feet', 'inches'], axis =1)

In [19]:
df['Value'] = df['Value'].astype(int)
df['Wage'] = df['Wage'].astype(int)
df['Release Clause'] = df['Release Clause'].astype(int)

In [20]:
df.head()

,ID,Name,Age,OVA,Nationality,Club,BOV,BP,Position,Player Photo,Club Logo,Flag Photo,POT,Team & Contract,foot,Growth,Joined,Loan Date End,Value,Wage,Release Clause,Contract,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,Gender,Weight (Kg),Height (cm),Year,Month,Day,Team,Date Started,Date Ended
0,2,G. Pasquale,33,69,Italy,Udinese,71,LWB,LM,https://cdn.sofifa.com/players/000/002/16_120.png,https://cdn.sofifa.com/teams/55/light_60.png,https://cdn.sofifa.com/flags/it.png,69,Udinese 2008 ~ 2016,Left,0,2008-07-01,NaN,625000,7000,0,2008 ~ 2016,313,75,50,59,71,58.0,338,73,65.0,60,69,71,347,68,74,68.0,69,68.0,347,74,68.0,69,68,68,320,72,69.0,63.0,66.0,50,NaN,208,70,69,69.0,56,14,5,15,10,12,1929,408,3 ★,2★,Medium,High,2 ★,71,59,70,71,68,69,4,65+0,65+0,65+0,68+0,67+0,67+0,67+0,68+0,68+0,68+0,68+0,69+0,69+0,69+0,69+0,69+0,71+-2,70+-1,70+-1,70+-1,71+-2,70+-1,69+0,69+0,69+0,70+-1,17+0,Male,82,182,2008,7,1,Udinese,2008,2016
1,16,Luis García,37,71,Spain,KAS Eupen,70,CM,CM CAM CDM,https://cdn.sofifa.com/players/000/016/19_120.png,https://cdn.sofifa.com/teams/2013/light_60.png,https://cdn.sofifa.com/flags/es.png,71,KAS Eupen 2014 ~ 2019,Right,0,2014-07-19,NaN,600000,7000,1100000,2014 ~ 2019,337,68,64,61,76,68.0,369,69,79.0,79,71,71,305,56,50,62.0,65,72.0,324,75,54.0,64,60,71,362,71,71.0,72.0,73.0,75,79.0,153,70,43,40.0,56,9,12,13,11,11,1906,385,4 ★,3★,Medium,Medium,1 ★,53,69,73,69,58,63,4,67+1,67+1,67+1,67+0,68+0,68+0,68+0,67+0,70+1,70+1,70+1,68+1,70+1,70+1,70+1,68+1,62+1,66+1,66+1,66+1,62+1,60+1,60+1,60+1,60+1,60+1,17+1,Male,64,177,2014,7,19,KAS Eupen,2014,2019
2,27,J. Cole,33,71,England,Coventry City,71,CAM,CAM RM RW LM,https://cdn.sofifa.com/players/000/027/16_120.png,https://cdn.sofifa.com/teams/1800/light_60.png,https://cdn.sofifa.com/flags/gb-eng.png,71,Coventry City 2016 ~ 2020,Right,0,2016-01-07,NaN,1100000,15000,0,2016 ~ 2020,337,80,64,41,77,75.0,387,79,84.0,77,69,78,295,48,42,71.0,59,75.0,284,72,58.0,29,56,69,317,69,39.0,69.0,74.0,66,NaN,99,35,34,30.0,51,9,6,13,16,7,1770,354,4 ★,4★,Medium,Low,2 ★,45,68,76,77,36,52,11,64+0,64+0,64+0,70+0,69+0,69+0,69+0,70+0,71+0,71+0,71+0,68+0,66+0,66+0,66+0,68+0,52+0,54+0,54+0,54+0,52+0,47+0,46+0,46+0,46+0,47+0,15+0,Male,73,175,2016,1,7,Coventry City,2016,2020
3,36,D. Yorke,36,68,Trinidad &amp; Tobago,Sunderland,70,ST,NaN,https://cdn.sofifa.com/players/000/036/09_120.png,https://cdn.sofifa.com/teams/106/light_60.png,https://cdn.sofifa.com/flags/tt.png,82,Sunderland 2009,Right,14,NaT,NaN,0,0,0,2009,264,54,70,60,80,NaN,255,68,NaN,46,64,77,176,59,62,NaN,55,NaN,239,63,NaN,51,66,59,271,59,70.0,72.0,NaN,70,NaN,75,34,41,NaN,68,5,21,64,21,21,1348,369,3 ★,1★,NaN,NaN,1 ★,61,66,66,69,47,60,3,67+0,67+0,67+0,66+0,67+0,67+0,67+0,66+0,70+0,70+0,70+0,66+0,68+0,68+0,68+0,66+0,56+0,65+0,65+0,65+0,56+0,57+0,51+0,51+0,51+0,57+0,22+0,Male,74,180,0,0,0,NaN,NaN,NaN
4,41,Iniesta,36,81,Spain,Vissel Kobe,82,CAM,CM CAM,https://cdn.sofifa.com/players/000/041/20_120.png,https://cdn.sofifa.com/teams/101146/light_60.png,https://cdn.sofifa.com/flags/es.png,81,Vissel Kobe 2018 ~ 2021,Right,0,2018-07-16,NaN,5500000,12000,7200000,2018 ~ 2021,367,75,69,54,90,79.0,408,85,80.0,70,83,90,346,61,56,79.0,75,75.0,297,67,40.0,58,62,70,370,58,70.0,78.0,93.0,71,89.0,181,68,57,56.0,45,6,13,6,13,7,2014,420,4 ★,4★,High,Medium,4 ★,58,70,85,85,63,59,149,72+3,72+3,72+3,79+0,79+0,79+0,79+0,79+0,82+-1,82+-1,82+-1,79+2,81+0,81+0,81+0,79+2,70+3,73+3,73+3,73+3,70+3,67+3,64+3,64+3,64+3,67+3,17+3,Male,68,170,2018,7,16

In [21]:
df.to_csv('/content/fifa21_cleaned.csv', index=False)